# Model 1

## Prepare data

In [1]:
#r "..\ConsoleApp1\bin\Debug\net6.0\ConsoleApp1.dll"

using System;
using ProjectOneClasses.Utilities;
using ProjectOneClasses;
using ConsoleApp1;

Data.LoadAllExampleData();

## Model

In [2]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using ProjectOneClasses.Utilities;

public class Model
{
    private List<double[]> trainedClusters;
    private List<int> trainedLabels;

    public void Train(List<double[]> data, int C, IReadOnlyDictionary<int, int> labels)
    {
        var labelsSet = new HashSet<int>(labels.Values);
        trainedLabels = labelsSet.ToList();
        for (int i = 0; i < labelsSet.Count; i++)
        {
            if (trainedLabels[i] < 0)
            {
                throw new Exception("Labels must not be negative");
            }
        }
        var labelsMapping = trainedLabels.ToImmutableDictionary(x => x, x => trainedLabels.IndexOf(x));
        var labelsTransformed = labels.ToImmutableDictionary(x => x.Key, x => labelsMapping[x.Value]);
        for (int i = labelsSet.Count; i < C; i++)
        {
            int label = -(i - labelsSet.Count + 1);
            trainedLabels.Add(label);
        }
        var fcm = new sSMC_FCM(data, C, labelsTransformed);
        fcm._solve();
        trainedClusters = fcm.Result.V.ToList();
    }

    public int Predict(double[] example)
    {
        var distances = trainedClusters.Select(x => Caculation.EuclideanDistanseSquared(x, example)).ToList();
        var minDistance = distances.Min();
        var minDistanceIndex = distances.IndexOf(minDistance);
        return trainedLabels[minDistanceIndex];
    }
}

## Test

### 100% Semi-supervised learning

In [3]:
using ProjectOneClasses.Utilities;

foreach (var dataInfo in Data.datas)
{
    var data = dataInfo.aCIDb;
    var X = data.X;
    var expect = data.expect;

    var indexes = Enumerable.Range(0, X.Count).ToList();
    indexes.Shuffle();

    // Config k - cross validation
    int pieTrainCount = 4;
    int pieTestCount = 1;
    int pieCount = pieTrainCount + pieTestCount;
    var pies = new IEnumerable<int>[pieCount];
    int pieSize = X.Count / pieCount;

    for (int i = 0; i < pieCount - 1; i++)
    {
        var pie = indexes.Skip(i * pieSize).Take(pieSize);
        pies[i] = pie;
    }
    pies[pieCount - 1] = indexes.Skip((pieCount - 1) * pieSize);

    double meanAccuracy = 0;
    int combinationCount = 0;

    foreach (var combination in Caculation.GenerateAllCombinations<IEnumerable<int>>(pies, Math.Min(pieTrainCount, pieTestCount)))
    {
        IEnumerable<int> trainIndexes = null;
        IEnumerable<int> testIndexes = null;
        int testIndexesIndex = 0;

        for (int i = 0; i < pieCount; i++)
        {
            if (testIndexesIndex < combination.Count && combination[testIndexesIndex] == pies[i])
            {
                testIndexes = testIndexes == null ? pies[i] : testIndexes.Concat(pies[i]);
                testIndexesIndex++;
            }
            else
            {
                trainIndexes = trainIndexes == null ? pies[i] : trainIndexes.Concat(pies[i]);
            }
        }

        var trainData = trainIndexes.Select(x => X[x]).ToList();
        var trainLabels = trainIndexes.Select((x, i) => new Tuple<int, int>(i, x)).ToImmutableDictionary(x => x.Item1, x => expect[x.Item2]);
        var testData = testIndexes.Select(x => X[x]).ToList();
        var testLabels = testIndexes.Select((x, i) => new Tuple<int, int>(i, x)).ToImmutableDictionary(x => x.Item1, x => expect[x.Item2]);

        var model = new Model();
        model.Train(trainData, data.C, trainLabels);
        var predictions = testData.Select(x => model.Predict(x)).ToList();
        var predictionsDict = predictions.Select((x, i) => new Tuple<int, int>(i, x)).ToImmutableDictionary(x => x.Item1, x => x.Item2);
        var accuracy = Caculation.Accuracy(testLabels, predictionsDict);

        meanAccuracy += accuracy;
        combinationCount++;
    }

    Console.WriteLine($"{dataInfo.Name.PadRight(22)} = {Math.Round(meanAccuracy / combinationCount * 100, 6)}%({data.C})");


}

Iris                   = 90%(3)
Iris Shuffled          = 90%(3)
Wine                   = 0.571429%(121)
Statlog(Heart)         = 19.62963%(2)
Vertebral Column(2C)   = 31.935484%(2)
Vertebral Column(3C)   = 39.354839%(3)
Glass                  = 49.047619%(6)
Bezdek Iris            = 90.666667%(3)
